In [0]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import shutil
import os
import requests
from typing import Tuple

In [0]:
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
predicted_20m = "predicted_gold_20m"
predicted_25m = "predicted_gold_25m"
predicted_30m = "predicted_gold_30m"
predicted_35m = "predicted_gold_35m"

In [0]:
def get_fylke_and_kommune(north: float, east: float) -> Tuple[int, int]:
    """Returns the fylke and kommune for a given coordinate"""
    url = f"https://api.kartverket.no/kommuneinfo/v1/punkt?nord={north}&ost={east}&koordsys=25833"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        fylke = data.get("fylkesnummer")
        kommune = data.get("kommunenummer")
        return fylke, kommune
    else:
        return None, None

In [0]:
local_gpkg = "/tmp/predicted.gpkg"
final_gpkg_dir = "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/geopackages/"
final_gpkg_path = os.path.join(final_gpkg_dir, "predicted.gpkg")

if os.path.exists(local_gpkg):
    os.remove(local_gpkg)

def append_layer(table, layer_name, first=False):
    df = spark.read.table(table)
    pdf = df.toPandas()
    pdf['geometry'] = pdf.apply(lambda row: Point(row['centroid_y'], row['centroid_x']), axis=1)
    fylke_kommune = pdf.apply(lambda row: get_fylke_and_kommune(row['centroid_y'], row['centroid_x']), axis=1)
    pdf['fylke'], pdf['kommune'] = zip(*fylke_kommune)
    pdf = pdf.drop(columns=['centroid_x', 'centroid_y'])
    gdf = gpd.GeoDataFrame(pdf, geometry='geometry')
    gdf.set_crs(epsg=25833, inplace=True)
    
    mode = "w" if first else "a"
    gdf.to_file(local_gpkg, driver="GPKG", layer=layer_name, mode=mode)

append_layer(predicted_20m, "predicted_20m", first=True)
append_layer(predicted_25m, "predicted_25m")
append_layer(predicted_30m, "predicted_30m")
append_layer(predicted_35m, "predicted_35m")

shutil.move(local_gpkg, final_gpkg_path)